In [1]:
sys.executable

'/home/goonieb/anaconda2/bin/python'

In [8]:
import os
import json
import pymongo
import musicbrainzngs
import arrow
import re
from datetime import datetime
from pymongo import MongoClient
from collections import defaultdict, OrderedDict

from dateutil import parser
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [17]:
DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
# setup mongo
client = MongoClient()
db = client["bandstour"]
venues =  db["venues"]
artists =  db["selectedArtists"]

def query_yes_no(question, default="yes"):
    """Ask a yes/no question via raw_input() and return their answer.
    "question" is a string that is presented to the user.
    "default" is the presumed answer if the user just hits <Enter>.
        It must be "yes" (the default), "no" or None (meaning
        an answer is required of the user).
    The "answer" return value is one of "yes" or "no".
    """
    valid = {"yes":True,   "y":True,  "ye":True,
             "no":False,     "n":False}
    if default == None:
        prompt = " [y/n] "
    elif default == "yes":
        prompt = " [Y/n] "
    elif default == "no":
        prompt = " [y/N] "
    else:
        raise ValueError("invalid default answer: '%s'" % default)

    while True:
        sys.stdout.write(question + prompt)
        choice = raw_input().lower()
        if default is not None and choice == '':
            return valid[default]
        elif choice in valid:
            return valid[choice]
        else:
            sys.stdout.write("Please respond with 'yes' or 'no' "\
"(or 'y' or 'n').\n")

def slugify(s):
    """
    Simplifies ugly strings into something URL-friendly.
    >>> print slugify("[Some] _ Article's Title--")
    some-articles-title
    """

    s = re.sub('\s+', ' ', s)
    s = s.replace(' ', '-')

    s = re.sub('\W+', ' ', s)

    s = s.strip()

    s = s.lstrip()
    s = re.sub('\W+', '_', s)
    return s

#WE WANT TO LOOP OVER ARTISTS IN ORDER TO DEDUPLICATE THE RECORDS AND ASSIGN TOURS TO ONE ENTITY ONLY.
#ex: "   dede" =add his dates to> "dede" who already has his.
#in case there is an ambiguity, we will have to choose
try:
    for artistA in artists.find() :


                #print "AAAA  =>", artistA["_id"]
                #print "AAAA slug  =>",slugify(artistA["_id"]).lower()
                for artistB in artists.find() :
                    if artistA["_id"]==artistB["_id"]:
                        pass
                    elif (fuzz.ratio(\
                                            slugify(artistA["_id"]).lower()\
                                            , slugify(artistB["_id"]).lower()\
                                           ) > 95 and len(artistA["gigs"]) >= len(artistB["gigs"])) :
                        print "AAAA  =>", artistA["_id"]
                        print "AAAA slug  =>",slugify(artistA["_id"]).lower()
                        print "AAAA gigs =>", len(artistA["gigs"])
                        print "AAAA mbid =>", artistA["mbid"]
                        print "BBBB  =>",artistB["_id"]
                        print "BBBB slug  =>",slugify(artistB["_id"]).lower()
                        print "BBBB gigs =>", len(artistB["gigs"])
                        print "BBBB mbid =>", artistB["mbid"]
                        print "_______________________________"
                        for gig in artistB["gigs"]:
                            artistA["gigs"].append(gig)
                        Aprime = artistA["gigs"]
                        #print Aprime

                        #Aprime2 = sorted(Aprime,key=lambda t: t["datetime"].strftime(DATETIME_FORMAT))


                        print "Aprime2",Aprime2
                        #Answer_art= query_yes_no("MERGE "+artistB["_id"]+ " WITH "+ artistA["_id"]+" ?" , default="no")
                        #print Answer_art
                        Answer_art = True
                        if Answer_art == True :
                            artistA["gigs"]=Aprime2
                            #print artistA["gigs"]
                            artists.update({"_id" :artistA["_id"]}, artistA, upsert=True, multi=False)
                            artists.remove({"_id" :artistB["_id"]})

    print "DONE!"
except:
    print "NOT FOUND ONE"
    

NOT FOUND ONE
